In [2]:
import os

In [3]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\muru1\\Muru-Car\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\muru1\\Muru-Car'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataCleaningConfig:
    root_dir: Path
    cleaned_data: Path
    unzip_data_dir: Path

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        ):

        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_cleaned_data_config(self) -> DataCleaningConfig:
        config = self.config.data_cleaning

        create_directories([config.root_dir])

        data_cleaning_config = DataCleaningConfig(
            root_dir=config.root_dir,
            cleaned_data=config.cleaned_data,
            unzip_data_dir = config.unzip_data_dir,
        )

        return data_cleaning_config

In [9]:
import os
from mlProject import logger
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, RobustScaler

In [12]:
class DataCleaning:
    def __init__(self, config: DataCleaningConfig):
        self.config = config


    def cleaned_data(self) -> pd.DataFrame:
      
        df = pd.read_csv(self.config.unzip_data_dir)

        
        df= df.sample(frac=1)
        maintenance_df = df.loc[df['Engine Condition'] == 1][:7218]
        great_df = df.loc[df['Engine Condition'] == 0]

        new_df = pd.concat([maintenance_df, great_df])

        new_df = df.sample(frac=1, random_state=42)

        save_path = os.path.join(self.config.root_dir, "new_df.csv")
        new_df.to_csv(save_path, index=False)

        return new_df

In [13]:
try:
    config = ConfigurationManager()
    data_cleaning_config = config.get_cleaned_data_config()
    data_cleaned = DataCleaning(config=data_cleaning_config)
    data_cleaned.cleaned_data()
except Exception as e:
    raise e

[2023-08-18 16:15:14,617: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-18 16:15:14,620: INFO: common: created directory at: artifacts]
[2023-08-18 16:15:14,623: INFO: common: created directory at: artifacts/data_cleaning]


In [14]:
pd.read_csv("artifacts/data_cleaning/new_df.csv")

,Engine rpm,Lub oil pressure,Fuel pressure,Coolant pressure,lub oil temp,Coolant temp,Engine Condition
0,551,2.341041,6.397153,1.839974,75.564105,75.167911,1
1,444,4.676569,5.658346,2.791732,82.940478,86.432827,1
2,665,4.329838,5.806792,4.760415,77.027646,76.311031,1
3,840,2.827211,10.715240,5.301226,80.642499,94.933593,1
4,706,3.234125,4.444432,2.907535,79.091705,85.886788,1
...,...,...,...,...,...,...,...
19530,2029,2.457721,5.886757,1.604229,78.404138,73.606039,0
19531,864,4.874107,8.162206,1.827879,81.266514,71.873059,1
19532,747,4.393084,6.710736,1.912920,87.331108,73.776276,0
19533,650,1.978674,5.576337,5.309385,77.590408,83.912826,1
